# Price stock tendency prediction pipeline

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import mlflow
import mlflow.sklearn
from tqdm.notebook import trange, tqdm

from DataHelper import *

# increase plot size
plt.rcParams["figure.figsize"] = (20,8)

sns.set_theme()

font = {'family' : 'DejaVu Sans', 'size'   : 25}

matplotlib.rc('font', **font)

### Features

In [ ]:
config = {
    'task':'classification',
    'threshold':None,
    'perform_grid_search':False,
    'param_grid':None,
    'use_class_weights':False
    'price_column':'Close',
    'predict_n':1
    
}